In [ ]:
import pandas as pd

In [ ]:
import numpy as np

On commence par charger les données qui sont dans un fichier .csv avec comme séparateur une ,

In [ ]:
file_name="dispersion_10240.csv"

In [ ]:
xy_train=pd.read_csv(file_name,header=0)

Visualisation

In [ ]:
print(xy_train)

Pour voir le format des données

In [ ]:
xy_train.shape

Certaines données sont catégorielles

In [ ]:
xy_train['model'] = xy_train['model'].astype('category')

In [ ]:
xy_train['ihaze'] = xy_train['ihaze'].astype('category')

In [ ]:
xy_train['clouds'] = xy_train['clouds'].astype('category')

In [ ]:
x_tot=np.array(xy_train[['cap','Mach','pitch','E_e_ai','power_setting','altitude','roll','yaw','hour','model','ihaze','hbase','hr','ta','vis','clouds']])

In [ ]:
y_tot=np.array(xy_train['IRS'])

**Modèle de type régression linéaire**

In [ ]:
import sklearn.linear_model as lm
linreg = lm.LinearRegression()

Il faut un vecteur de type (10240,1) pour l'apprentissage, on fait un reshape

In [ ]:
y_tot=y_tot.reshape(10240,1)

In [ ]:
lr=linreg.fit(x_tot, y_tot)

Si on veut normaliser les données

In [ ]:
from sklearn import preprocessing

Régression linéaire avec input normalisés et passage au log sur la sortie

In [ ]:
x_scaled = preprocessing.scale(x_tot)

In [ ]:
y_tot=log(y_tot)

In [ ]:
lr=linreg.fit(x_scaled, y_tot)

In [ ]:
lr.coef_

In [ ]:
linreg.score(x_scaled,y_tot)

Tracés

In [ ]:
%pylab inline

In [ ]:
plt.plot(y_tot,lr.predict(x_scaled),'o')

**Modèle de type MLP**

On sépare les données entre apprentissage et test (60%)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y_tot, test_size=0.6)

In [ ]:
from sklearn.neural_network import MLPRegressor

In [ ]:
nnet=MLPRegressor(hidden_layer_sizes=(11,),max_iter=5000,activation='tanh',alpha=0.05,solver='lbfgs')

In [ ]:
y_train=y_train.reshape(4096,)

In [ ]:
y_test=y_test.reshape(6144,)

In [ ]:
nnet.fit(x_train,y_train)

In [ ]:
y_pred=nnet.predict(x_test)

In [ ]:
%pylab inline

In [ ]:
plt.plot(y_test, y_pred,'o')

Calcul du coefficient de détermination R2

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(y_test,y_pred)

**Exercice 1** : Tester avec d'autres paramètres

**Exercice 2** : Tester d'autres métamodèles